In [7]:
import pandas as pd
from pathlib import Path

In [8]:
current_file_path = Path(__vsc_ipynb_file__)
main_directory = current_file_path.parents[1]
data_directory = main_directory.joinpath('data/template_model/Template_Model_1.csv')
stg_code_path = main_directory.joinpath('data/databases/transportation/stg_code_dummy.csv')
distances_path = main_directory.joinpath('data/databases/transportation/distances_dummy.csv')
scenarios_path = main_directory.joinpath('data/databases/transportation/scenarios_dummy.csv')
modes_path = main_directory.joinpath('data/databases/transportation/modes_dummy.csv')
mode_scenario_path = main_directory.joinpath('data/databases/transportation/mode_scenario_dummy.csv')
trans_mode_path = main_directory.joinpath('data/databases/transportation/trans_mode_dummy.csv')

In [9]:
df = pd.read_csv(data_directory)

In [10]:
trans_mass = df[df['Life Cycle Stage'] == '[A1-A3] Product']

In [11]:
trans_mass = trans_mass[['element_index', 'Material Group', 'Mass Total (kg)']]

In [12]:
stg_code = pd.read_csv(stg_code_path)
distances = pd.read_csv(distances_path)
scenarios = pd.read_csv(scenarios_path)
modes = pd.read_csv(modes_path)
mode_scenario = pd.read_csv(mode_scenario_path)
trans_mode = pd.read_csv(trans_mode_path)

In [13]:
mass = (trans_mass
    .merge(
        stg_code,
        how='right',
        left_on='Material Group',
        right_on='material_id'
    ).drop(
        columns='material_id'
    )
)

In [14]:
mode_scenario_df = (
    mode_scenario.merge(
        modes,
        left_on='mode_id',
        right_on='id',
        suffixes=(None, '_mode')
    ).merge(
        scenarios,
        left_on='scenario_id',
        right_on='id',
        suffixes=(None, '_scenario')
    )
)

In [15]:
mode_scenario_df = mode_scenario_df[['id', 'name', 'name_scenario']].rename(columns={'name': 'mode_name', 'name_scenario': 'scenario_name'})

In [16]:
trans_mode_df = (
    trans_mode.merge(
        mode_scenario_df,
        left_on='mode_scenario_id',
        right_on='id',
        suffixes=(None, '_mode_scenario')
    )
)

In [17]:
trans_mode_df = trans_mode_df[['mode_scenario_id', 'return_trip_factor', 'gwp', 'mode_name', 'scenario_name']]

In [18]:
trans_mode_df

,mode_scenario_id,return_trip_factor,gwp,mode_name,scenario_name
0,1,1.25,2.0,rail,domestic
1,2,1.25,8.0,truck,domestic
2,3,1.25,2.4,rail,global
3,4,1.50,16.2,truck,global


In [19]:
mass_distance = mass.merge(
    distances,
    how='left',
    on = 'stg_code'
)

In [20]:
final = mass_distance.merge(
    trans_mode_df,
    on = 'mode_scenario_id'
).drop(columns=['id', 'mode_scenario_id', 'reference'])

Transportation impacts = mass * distance * EF * RF

In [21]:
final = final.assign(total_a4=final['Mass Total (kg)'] * final['distance'] * final['gwp'] * final['return_trip_factor'])

In [22]:
final

,element_index,Material Group,Mass Total (kg),stg_code,distance,return_trip_factor,gwp,mode_name,scenario_name,total_a4
0,Element_0,Metal,2217.515116,1,30,1.25,2.0,rail,domestic,1.663136e+05
1,Element_0,Metal,2217.515116,1,60,1.25,8.0,truck,domestic,1.330509e+06
2,Element_0,Metal,2217.515116,1,90,1.25,2.4,rail,global,5.987291e+05
3,Element_0,Metal,2217.515116,1,120,1.50,16.2,truck,global,6.466274e+06
4,Element_4,Metal,12508.117480,1,30,1.25,2.0,rail,domestic,9.381088e+05
...,...,...,...,...,...,...,...,...,...,...
131,Element_32,Plaster,19056.464290,9,1080,1.50,16.2,truck,global,5.001178e+08
132,Element_33,Coating,366.306357,10,300,1.25,2.0,rail,domestic,2.747298e+05
133,Element_33,Coating,366.306357,10,600,1.25,8.0,truck,domestic,2.197838e+06
134,Element_33,Coating,366.306357,10,900,1.25,2.4,rail,global,9.890272e+05
